# Import libraries

In [1]:
import pandas as pd
import numpy as np
import nltk

from collections import Counter

from sklearn import clone

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.metrics import accuracy_score

%matplotlib inline

# Import dataset

In [2]:
df = pd.read_csv('../../Data/tribe_dynamics_data.csv')
df.head(1)

,Unnamed: 0,Unnamed: 0.1,brand_id,worker_id,mturker,post_hash,answer,date,duration_seconds,text,model_decision,timestamped_model,lang
0,0,0,18792,TRIBE_103_allayne.low@gmail.com,False,3ca62dcea583b7aa204fc52fe6b2826c,False,2017-07-16,NaN,・\r\nケイト スペード ニューヨークの\r\n2017Fall Collectionに招...,False,NaN,ja


In [3]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df.head(1)

,brand_id,worker_id,mturker,post_hash,answer,date,duration_seconds,text,model_decision,timestamped_model,lang
0,18792,TRIBE_103_allayne.low@gmail.com,False,3ca62dcea583b7aa204fc52fe6b2826c,False,2017-07-16,NaN,・\r\nケイト スペード ニューヨークの\r\n2017Fall Collectionに招...,False,NaN,ja


In [4]:
df_small = df[['text','lang','model_decision','answer','mturker']]
df_small.head(3)

,text,lang,model_decision,answer,mturker
0,・\r\nケイト スペード ニューヨークの\r\n2017Fall Collectionに招...,ja,False,False,False
1,青春日記【挨拶のハグじゃダメなの】\r\n僕が中学3年生の時だ。\r\n高山私立松倉中学校と...,ja,False,False,False
2,fabric&thingsで本日のみ開催しております\r\nいちごのプチレストラン La F...,ja,False,False,False


### Extract posts of just one language

In [5]:
language = 'en' # Change here to test a different language
lang_df = df_small[df_small.lang == language]
lang_df.head(3)

,text,lang,model_decision,answer,mturker
67,"(near) PORTLAND, OR: Shye Girl and Joey are at...",en,False,False,False
68,Remember 16 year old Daisy (center) who was fo...,en,False,False,False
69,"(near) PORTLAND, OR: 7 year old Maia was so cl...",en,False,False,False


In [6]:
lang_df.shape

(9426, 5)

In [7]:
# Remove perfect duplicates
lang_no_dup_df = lang_df.drop_duplicates(keep='first', inplace=False)
lang_no_dup_df.shape

(2610, 5)

In [8]:
print('Dropped {} completely equal samples'.format(lang_df.shape[0]-lang_no_dup_df.shape[0]))

Dropped 6816 completely equal samples


# Models computation

In [9]:
# Array with all posts text
texts_arr = lang_no_dup_df.text.values

In [10]:
# 'Ground truth' labels
y_true = lang_no_dup_df.answer.values
# Tribe's predicted labels
y_tribe = lang_no_dup_df.model_decision.values

In [11]:
print("Dataset balance:")
print("True labels: {}".format(Counter(y_true)[True]))
print("False labels: {}".format(Counter(y_true)[False]))

Dataset balance:
True labels: 299
False labels: 2311


## Bag of words

In [12]:
# Bag of words
#count_vect = CountVectorizer(stop_words='english')
#count_vect = CountVectorizer(ngram_range=(1,2))
count_vect = CountVectorizer()
X_bag = count_vect.fit_transform(texts_arr)
X_bag.shape

(2610, 37827)

In [13]:
# TF-IDF on the vectorized dataset
X_tfidf = TfidfTransformer().fit_transform(X_bag)
X_tfidf.shape

(2610, 37827)

If you have fairly little data and you are going to train a supervised classifier, then machine learning theory says you should stick to a classifier with high bias. For example, there are theoretical and empirical results that Naive Bayes does well in such circumstances, although this effect is not necessarily observed in practice with regularized models over textual data. At any rate, a very low bias model like a nearest neighbor model is probably counterindicated. Regardless, the quality of the model will be adversely affected by the limited training data.

In [14]:
random_seed = 231

classifiers = [
    ('Naive Bayes', MultinomialNB()),
    #('Gaussian Naive Bayes', GaussianNB()), # only works with dense matrices
    ('Logistic Regression (no reg.)', LogisticRegression(C=1e6, random_state=random_seed, n_jobs=-1)),
    ('Logistic Regression (some reg.)', LogisticRegression(C=1e-2, random_state=random_seed, n_jobs=-1)),
    ('Linear SVM (no reg.)', LinearSVC(C=1e6, random_state=random_seed)),
    ('Linear SVM (some reg.)', LinearSVC(C=1e-2, random_state=random_seed))
    ]

In [15]:
# Stratified CV folds
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_seed)

In [16]:
print('Bag of words - 10-fold cross validation accuracy:')

for (model_name,clf) in classifiers:    
    scores = cross_val_score(clone(clf), X_bag, y_true, cv=folds, scoring='accuracy', n_jobs=-1)
    print('{}: \t {:.2f} +/- {:.2f}'.format(model_name, np.mean(scores), np.std(scores)))

Bag of words - 10-fold cross validation accuracy:
Naive Bayes: 	 0.83 +/- 0.02
Logistic Regression (no reg.): 	 0.83 +/- 0.02
Logistic Regression (some reg.): 	 0.88 +/- 0.01
Linear SVM (no reg.): 	 0.75 +/- 0.07
Linear SVM (some reg.): 	 0.86 +/- 0.02


In [17]:
print('TF-IDF - 10-fold cross validation accuracy:')

for (model_name,clf) in classifiers:    
    scores = cross_val_score(clone(clf), X_tfidf, y_true, cv=folds, scoring='accuracy', n_jobs=-1)
    print('{}: \t {:.2f} +/- {:.2f}'.format(model_name, np.mean(scores), np.std(scores)))

TF-IDF - 10-fold cross validation accuracy:
Naive Bayes: 	 0.89 +/- 0.00
Logistic Regression (no reg.): 	 0.83 +/- 0.01
Logistic Regression (some reg.): 	 0.89 +/- 0.00
Linear SVM (no reg.): 	 0.81 +/- 0.05
Linear SVM (some reg.): 	 0.89 +/- 0.00


In [18]:
print("Tribe's model accuracy:\t{:.2f}".format(accuracy_score(y_true, y_tribe)))

Tribe's model accuracy:	0.92


# PCA plots